# 1. Questions:

#### Define the problem. What is the input? What is the output? What data mining function is required? What could be the challenges? What is the impact? What is an ideal solution?

    - The problem: predict if a specific reddit post is fake news or not, by looking at its title. 
    - The input: news posts
    - the output: if news post is fake or not based on its title.
    - Hence the output is fake or not then the data mining function: is binary classification
    - Challenges: Problems in spelling, a lot of stop words, unbalanced labels
    - the impact: detecting if news is fake or not so we can prevent it before people can see it.
    - the solution: is to do tokenazation and count words then give them weights based on their occurances then use some machine learning model like random forest or xgboost

#### What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?

    - An n-gram is a contiguous sequence of n items from a given sample of text or speech. The items can be characters, words, or other units of text. A character n-gram is a sequence of n characters from a given text. For example, the 2-character (or bigram) n-grams of the word “text” are “te”, “ex”, and “xt”.A word n-gram is a sequence of n words from a given text. For example, the 2-word (or bigram) n-grams of the sentence “I love to read” are “I love”, “love to”, and “to read”.
    
    - The OOV (out-of-vocabulary) issue refers to the problem of encountering words or tokens that were not seen during training. Word n-grams tend to suffer more from the OOV issue than character n-grams. This is because the vocabulary of word n-grams is typically much larger than the vocabulary of character n-grams, and it is more likely that a new word will be encountered that was not seen during training. In contrast, the vocabulary of character n-grams is limited to the set of possible characters in the language, which is typically much smaller.

#### What is the difference between stop word removal and stemming? Are these techniques language-dependent?

    - Stop word removal and stemming are two different techniques used in natural language processing. Stop word removal involves removing common words that do not carry much meaning, such as “the”, “is”, “and”, etc. Stemming involves reducing words to their root form, for example, “running” would be reduced to “run”. These techniques can be language-dependent as stop words and word forms vary between languages.
    Yes, these techniques can be language-dependent as stop words and word forms vary between languages. For example, the list of stop words used for English text would be different from the list used for French text. Similarly, stemming algorithms are often designed for specific languages and may not work as well for other languages.

####  Is tokenization techniques language dependent? Why?

- Yes, tokenization techniques can be language-dependent. Tokenization is the process of breaking down a piece of text into small units called tokens. A token may be a word, part of a word or just characters like punctuation. Every language has its own grammatical constructs, which are often difficult to write down as rules. These issues of tokenization are language-specific and require the language of the document to be known.

#### What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?

- CountVectorizer and TfidfVectorizer are both methods used in natural language processing to vectorize text. CountVectorizer simply counts the number of times a word appears in a document (using a bag-of-words approach), while TfidfVectorizer takes into account not only how many times a word appears in a document but also how important that word is to the whole corpus. This is done by penalizing words that often appear across all documents, reducing the count of these as these words are likely to be less important. It would not be feasible to use all possible n-grams as the number of possible n-grams grows exponentially with the length of the n-grams. Instead, it is common to select a range of n-gram lengths (e.g., unigrams, bigrams, and trigrams) and only consider n-grams within that range.Would you like more information on this topic?

In [3]:
#! pip install xgboost


In [10]:
 # importing important liabraries
from IPython.display import display_html
import matplotlib.gridspec as gridspec
from scipy.stats import gaussian_kde
from nltk.corpus import stopwords
from itertools import chain,cycle
from collections  import Counter
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
import nltk
from sklearn.model_selection import GridSearchCV


import string
import re
from nltk.stem import WordNetLemmatizer

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn import svm
# WORDNET LEMMATIZER (with appropriate pos tags)
nltk.download('punkt')
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# 2. Data Preprocessing

In [11]:
# the dictonaries below are words that we gonna handdle and replace its abbrevaiton by the complete word
contractions = {"'re":'are',"u're":"you are","they've":"they have",
                "i'm":"i am","you've":"you have","we've":"we have","wouldn't":'would not',
                "couldn't":'could not',"isn't":'is not',"don't":'do not',"haven't":'have not',
                "hasn't":'has not',"doesn't":'does not',"I've":'i have',"aren't":'are not',
                "shouldn't":'should not',"can't":'can not',"hadn't":'had not',"wasn't":'was not',
                "won't":'will not',"there's":'there is',"didn't":'did not',"weren't":'were not',
                "that's":'that is',"he's":'he is',"she's":'she is',"they're": 'they are',
                "cya":'see you',"we're":'we are',"it's":'it is',"you're":"you are","i've":'i have',
                "wtf":'what the fuck',"must've":'must have',"should've":"should have","would've":"would have",
                "could've":"could have","how're":'how are',"it's":'it is',"what's":'what is',"\\":"","\\n":"","cuz":"because", "cant": "can not"}
numer_words = {'sherfield72':'mention','rs40000cr':'mention','air1bullet':'air bullet','960kzim':'mention',
               "'revel8ion":'revelation',"'news786":'news 786',"'mumbai24x7":'mention','17months':'17 months',
               'ny1burst':'new york burst','tkyonly1fmk':'mention','nikoniko12022':'mention','hamosh84':'mention',
               'ibliz140':'mention','bkb066gp':'mention','tksgs0810':'mention','mhtw4fnet':'mention',"'psalm34":'mention',
               "76mins":'76 minutes',"oliviamiles01":'mention','friend50':'friend 50', "'master0fsloths":'master of sloths',
               "'s3xleak":'sex leak','push2left':'push to left',"'411naija":'mention','wsvr1686b':'mention','911bombing':'911 bombing',
               "fuckjf9jjzs',":'mention','b4federal':'mention','7amdollela':'7 am dollela','3million': '3 million','vaping101':'mention',
               "'matako_3":'mention','150bilno':'mention',"'neil_eastwood77":'mention','hirochii0':'mention',"20mins',":'20 mins',
               "'chickmt123":'mention','drayesha4':'mention','death2usa':'death to usa',"2gether',":'together','time2015':'time 2015',
               "'nflweek1picks":'mention','martinmj22':'mention',"'lizzie363":'mention','russaky89':'mention','friend59':'mention',
               "you5jbwgge',":'mention'}

word_numbers = {'06jst':'', '0npzp':'', '10x':'', '12jst':'', '12news':'', '12v':'', '18jst':'', '1x1':'', '24v':'', '325ci':'', 
                '3x':'', '3x5':'', '429cj':'', '43c':'', '4x4':'', '53inch':'', '6c':'', '712c':'', '_':'', '__':'', '_ah':'', 
                '_deo':'', '_keits':'', '_that':'', '_turns':'',"',":"","sherfield72":"mention","pemantaujkt48":"mention",
                "bracelet10mm":"mention", "'chickmt123":"mention","'tarmineta3":"mention","you5jbwgge',":'mention',
                "'mumbai24x7":'mention','5sosglobalsquad':'mention','dylanmcclure55':'mention','tanstaafl23':'',
                'footballfreestyle24':'football free style 24',"40hourfamine":"40 hour famine","15000270364',":'number',
                'dylanmcclure55':'mention ',"'master0fsloths":'master of sloths','insurers163':'insurers 163',"15000270653',":'number',
                'zonewolf123':'mention',  "'tkyonly1fmk":'mention', "sherfield72'":'mention ', "8437150124',":'number',
                '40hourfamine':'40 hour famine','watchmanis216':'mention','tarmineta3':'mention',"'neil_eastwood77":'mention',
                "oliviamiles01',":'mention','hyider_ghost2':'mention ','rabidmonkeys1':'mention ','since1970the':'since 1970 the',
                "'nflweek1picks":'mention','then0mads0ul':'mention',"fuckjf9jjzs',":'','janenelson097':'mention','087809233445':'0',
                'onlinemh370':'mention', '2slow2report':'to slow to report',"'nikoniko12022":'mention', 'shawie17shawie':'mention'}

repetead_vowels = {"soooo":"so","baaaack":"back","aaaaaaallll":"all","sparxxx":"mention",
                   "oooh":"oh","lmfaooo":"laughing my fat ass off","fuckkkkkk":"fuck","vuuuuu":"vu",
                   "mxaaaa":"mention","caaaaaall":"call","goooooooaaaaaal":"goal",'shoook':'shook',
                   'maaaaan':'man','sooooooo':'soon','blaaaaaaa':'bla','youuu':'you','yeeessss':'yes',
                   'noooo':'no','totoo':'totoo',"ooohshit":'oh shit','nooo':'no','wompppp':'womp','maddddd':'mad',
                   "'thankkk":"thank","plsss":"please",'ohhhh':'oh',"'xoxoxxxooo":'hugs and kisses',
                   "'damnnnn":"damn","'damnnnn":"damn","ooo":"oo","oooo":"oo","ooooo":"oo",
                   "ssssss":"ss","ssss":"ss","yyyy":'y',"yyyy":'y',"lll":" ll ","eee":"ee",
                   "wwwww":"w","aaa":"a","nnn":"n","rrr":"r","aaaaaa":"a","gggg":"g",
                   "wwww":"w","www":"w","mmmm":"m","iiii":"i","iii":"i","sss":'s',"yyy":"y",
                   "uuu":"u","eee":'e','hhh':'h',"ddd":"d",
                   "zzzz',":"sleep","xxx',":"","mhmmm":"hmm","hmmm":'hmm', '"': ''}


In [17]:
nltk.download('stopwords') # download stop words


def cont_to_exp(x, list_): #Used to expand contractions such has I'm to I am 
    x = str(x).lower()
    if type(x) is str:
        for key in list_:
            value = list_[key]
            x = x.replace(key,value)
        return x
    else :
        return x
    
# the function below replace links and mentions with `link` or 'mention' word    
def Prepro_1(text): 
    Ssymbols = ["\x89","\x9d"]    
    ''' Lower Replace #@mentions with Mention to save that pattern'''
    x = re.sub(r'(https?://[^\s]+)'," LINK ",text) #Link
    x = re.sub(r'\S*(\w+)(_)(\w+)\S*',"MENTION",x) #Mention
    x = re.sub(r'(?<![@\w])@(\w{1,25})'," MENTION ",x) #Mention
    x = re.sub(r"#(\w+)\S"," MENTION ",x) #MENTION
    x = re.sub(r"[âãåçèêìïñòó÷ûüª©¨¤«£¢¬`\...¡!¼#\$%&\()\*\+,-./:;<=>?@\[\]^_`{|}~,]+"," ", x)
    x = re.sub(r"'(?!s)","", x)
    x = re.sub(r"\\n|\\n\\"," " ,x) # valdiate those spaces /n that python was not able to decode.
    x = x.lower() # make it lower case.
    x = re.sub(str(Ssymbols)," ",x)
    return x.lower()
# the function below do lemmatization
def lematization(corpus):
    lemmatizer = nltk.WordNetLemmatizer() # create lemmatizer object
    NEC = [word for word in corpus.split() if word.lower() not in stopwords.words('english')] # removing stop words
    lematized_words = [] # list to save lemmatized words
    for word in NEC:
        verb = lemmatizer.lemmatize(word, 'v') # lemmatize verbs
        adj = lemmatizer.lemmatize(word, 'a')  # lemmatize words
        none = lemmatizer.lemmatize(word, 'n')  # lemmatize nones
        adv = lemmatizer.lemmatize(word, 'r')  # lemmatize nones
        if verb != word: # if the verb variable changed 
            lematized_words.append(verb)
        elif adj != word:
            lematized_words.append(adj)
        elif none != word:
            lematized_words.append(none)
        elif adv != word:
            lematized_words.append(adv)
        else:
            lematized_words.append(word)
            
    return ' '.join(lematized_words)
    #return ' '.join([lemmatizer.lemmatize(word, 'v') for word in NEC])
    
# Define function to slice text
def slice_text(text, max_words=25):
    words = text.split()
    sliced_words = words[:max_words]
    return ' '.join(sliced_words)    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
train_data = pd.read_csv('xy_train.csv') # reading the dataframe.
train_data.head()

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0


In [20]:
# the function below do some preprocessing and clean the data
def data_preprocessing(data):
    data['text'] = data['text'].apply(slice_text) # get first 50 words of the text
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-z ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-z]\b", re.IGNORECASE)
    RE_ASCII = re.compile(r"[^A-Za-z,.!? ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-z,.!?]\b", re.IGNORECASE)
    data['text_'] = data['text'].apply(lambda x: re.sub(RE_TAGS," " ,x)) ##Removing spac
    data['text_'] = data['text'].apply(lambda x: re.sub(RE_ASCII, " ", x)) ##Removing spac
    data['text_'] = data['text'].apply(lambda x: re.sub(RE_SINGLECHAR, " ", x)) ##Removing spac
    data['text_'] = data['text'].apply(lambda x: re.sub(RE_WSPACE, " ", x)) ##Removing spac
    data['text_'] = data['text'].apply(lambda x: re.sub(r"\\|\n|\\\n|\r|\t"," " ,x)) ##Removing spaces
    data['text_'] = data.text_.apply(lambda x: cont_to_exp(x,contractions)) # expand contractions such has I'm to I am 
    # test_['text_'] = test_.text_.apply(lambda x: cont_to_exp(x,contractions)
    data['text_'] = data.text_.apply(lambda x: Prepro_1(x)) # replace links and mentions by MENTION or LINK words
    data['text_'] = data.text_.apply(lambda x: cont_to_exp(x,numer_words)) #sperate numbers form words
    data['text_'] = data.text_.apply(lambda x: cont_to_exp(x,word_numbers))
    data['text_'] = data.text_.apply(lambda x: cont_to_exp(x,repetead_vowels))
    data['text_'] = data.text_.apply(lambda x: re.sub(r"\b\d+\b", " number ",x))
    data['text_'] = data.text_.apply(lambda x: re.sub(r"\b([0-9+]+[rd|th|nd]+)\b", " number ",x)) # replace numbers by word `number`
    data['text_'] = data.text_.apply(lambda x: re.sub(r"([0-9]+[am|pm|hr|s|yr|yrs|hours|day|days|years|yesterday|year])", " time ",x)) # replace time by word `time`
    data['text_'] = data.text_.apply(lambda x: re.sub(r"([0-9]+[fps|mm|km|w|ft|oz|lbs|whts|kg]+)"," measure ",x)) # replace any measure by word `measure`
    data['text_'] = data.text_.apply(lematization) # do lemmatization

    return data

In [21]:
train_data = data_preprocessing(train_data) # apply preprocessing

In [22]:
train_data.head()

,id,text,label,text_
0,265723,A group of friends began to volunteer at a hom...,0,group friend begin volunteer homeless shelter ...
1,284269,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister mention nerve attack fo...
2,207715,"In 1961, Goodyear released a kit that allows P...",0,number goodyear release kit allow p time bring...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0,happy birthday bob barker price right host hed...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0,obama nation 聙innocent cop unarm young black m...


In [23]:
train_data['label'].unique() # print labels

array([0, 1, 2], dtype=int64)

- we have 3 labels so we gonna convert label 2 to label 1

In [24]:
train_data['label'] = train_data['label'].apply(lambda x: 1 if x == 2 else  x) # convert label 2 to label 1

In [25]:
train_data['label'].unique() # check if label changed

array([0, 1], dtype=int64)

In [26]:
from bokeh.models import NumeralTickFormatter
# Word Frequency of most common words
word_freq = pd.Series(" ".join(train_data["text_"]).split()).value_counts() # count words after preprocessing
word_freq[1:40]

year        3761
get         3431
make        3262
one         3001
time        2924
find        2880
new         2854
say         2795
like        2729
look        2663
man         2551
take        2433
trump       2355
u           2308
colorize    2290
old         2255
first       2129
use         2101
people      2056
go          2045
poster      1904
woman       1737
day         1732
war         1700
see         1619
give        1523
know        1466
post        1443
show        1442
leave       1420
american    1418
work        1410
life        1357
world       1355
think       1330
come        1294
鈥           1290
state       1259
school      1257
dtype: int64

In [27]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq") # lowest frequented words

,index,freq
0,cartwright,1
1,ground鈥,1
2,鈥楨lephant鈥檚,1
3,anoddsort,1
4,2000脳1333,1
5,btech,1
6,scorese,1
7,鈥渂right,1
8,kal,1
9,蜆蜄虡太o汀虩蛧蛶蛧虨贪f痰贪太蛵虪虧虡虡,1


In [28]:
len(word_freq) # length of words

43438

In [29]:
# Distribution of ratings
train_data["label"].value_counts(normalize=True) # check of the data balanced or not.

0    0.5362
1    0.4638
Name: label, dtype: float64

In [30]:
# Sample data - 20% of data to test set
train, test = train_test_split(train_data, test_size=0.20, shuffle=True)

X_train = train["text_"]
Y_train = train["label"]
X_test = test["text_"]
Y_test = test["label"]
print(X_train.shape) # print shape of train
print(X_test.shape) # print shape of test

(48000,)
(12000,)


In [31]:
# the two lines of code below define a vaildation set to use in grid search instead of cross vaildation.
split_index = [-1 if x in X_train.index else 0 for x in train_data.index]
pds = PredefinedSplit(test_fold = split_index) 

# 3. Modelling

## 3.1. Random Forest

#### 3.1.1 using word analyzer

In [36]:
# feature creation and modelling in a single function
pipe_rf = Pipeline([("tfidf", TfidfVectorizer()), ("rf", RandomForestClassifier())])

# define parameter space to test 
rf_params = {
    'tfidf__analyzer':["word"], # 1
    'tfidf__max_df':[0.8, 0.9, 0.95], # 4
    'tfidf__min_df':[1, 2, 5, 10], # 4
    #'tfidf__max_features':[4000, 5000, 6000], # 3
    'tfidf__norm':["l2"], # 1
     'tfidf__ngram_range': [(1, 2)], # 2
     'tfidf__stop_words': ['english'], # 1
     'rf__n_estimators':[100], # 2
     #'rf__max_depth': [1000, 2000] # 2
}


pipe_rf = RandomizedSearchCV(pipe_rf, rf_params, verbose=100, cv=pds, scoring="roc_auc", refit=True, n_iter=10)
pipe_rf.fit(train_data['text_'], train_data['label']) # fit the random forest model


Fitting 1 folds for each of 10 candidates, totalling 10 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=10, tfidf__max_df=0.8, tfidf__analyzer=word, rf__n_estimators=100 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=10, tfidf__max_df=0.8, tfidf__analyzer=word, rf__n_estimators=100, score=0.854, total= 1.7min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.9, tfidf__analyzer=word, rf__n_estimators=100 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.9, tfidf__analyzer=word, rf__n_estimators=100, score=0.854, total= 1.9min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.7min remaining:    0.0s
[CV] tfidf__stop_wor

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0, ..., -1, -1])),
                   error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('tfidf',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                      

In [37]:
pipe_rf.best_score_ # print best score

0.8579876728558414

- Best score of random forest is 85.7% this is the first trial using those hyperparameters which we assumed they are good for this problem so we gonna compare this result with different hyperparamters and different models in the upcomming trials

In [38]:
pipe_rf.best_params_ # print best parameters

{'tfidf__stop_words': 'english',
 'tfidf__norm': 'l2',
 'tfidf__ngram_range': (1, 2),
 'tfidf__min_df': 2,
 'tfidf__max_df': 0.9,
 'tfidf__analyzer': 'word',
 'rf__n_estimators': 100}

### 3.1.2 Reading and predict the test data

In [103]:
test_data_orig = pd.read_csv('x_test.csv') # reading the test data
test_data = data_preprocessing(test_data_orig.copy()) # do the preprocessing

submission = pd.DataFrame() # create a dataframe
submission['id'] = test_data_orig['id'] # get the id column

submission['label'] = pipe_rf.predict_proba(test_data['text_'])[:,1] # predict the values

submission.to_csv('sample_submission_walkthrough.csv', index=False) # save the result

#### 3.1.3 using character analyzer

In [51]:
# feature creation and modelling in a single function
pipe_rf = Pipeline([("tfidf", TfidfVectorizer()), ("rf", RandomForestClassifier())])

# define parameter space to test 
rf_params = {
    'tfidf__analyzer':["char"], # 1
    'tfidf__max_df':[0.8, 0.9], # 2
    'tfidf__min_df':[1, 2, 5, 10], # 4
    #'tfidf__max_features':[4000, 5000, 6000], # 3
    'tfidf__norm':["l2"], # 1
     'tfidf__ngram_range': [(1, 2), (1, 3), (1, 4)], # 3
     'tfidf__stop_words': ['english'], # 1
     'rf__n_estimators':[100], # 2
     #'rf__max_depth': [1000, 2000] # 2
}


pipe_rf_w = RandomizedSearchCV(pipe_rf, rf_params, verbose=100, cv=pds, scoring="roc_auc", refit=True, n_iter=15)
pipe_rf_w.fit(train_data['text_'], train_data['label']) # fit the random forest model


Fitting 1 folds for each of 15 candidates, totalling 15 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=1, tfidf__max_df=0.8, tfidf__analyzer=char, rf__n_estimators=100 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=1, tfidf__max_df=0.8, tfidf__analyzer=char, rf__n_estimators=100, score=0.750, total= 4.2min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.2min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=2, tfidf__max_df=0.9, tfidf__analyzer=char, rf__n_estimators=100 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=2, tfidf__max_df=0.9, tfidf__analyzer=char, rf__n_estimators=100, score=0.751, total= 2.8min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.1min remaining:    0.0s
[CV] tfidf__stop_words

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0, ..., -1, -1])),
                   error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('tfidf',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                      

In [56]:
pipe_rf_w.best_score_

0.8266638598844015

- The trial above using char analyzer shows that it is worse than using word analyzer as it gets 82%

## 3.2 Xgboost

In [52]:
Pipe_ = Pipeline([('tfidf', TfidfVectorizer()), ('Model', xgb.XGBRegressor())]) # pipline for xgboost
# xgboost parameters
params = {
        'tfidf__analyzer':["word"], # 1
        'tfidf__max_df':[0.5, 0.8, 0.99], # 3
        'tfidf__min_df':[1, 2, 5, 10], # 4
        #'tfidf__max_features':[2500 ,3000, 3500, 4000, 4500, 5000, 6000, 8000, 10000], # 5
        'tfidf__norm':["l2"], # 1
        'tfidf__ngram_range': [(1, 2)], # 2
        'tfidf__stop_words': ['english'], # 1
        #'Model__min_child_weight': [1, 5, 10], # 3
        #'Model__gamma': [0.5, 1, 1.5, 2, 5], # 5

        #'Model__max_depth': [100,200,300,400,500,1000,2000] # 7
        }
xgb_gs = RandomizedSearchCV(Pipe_, params, cv = pds, verbose =100, scoring='roc_auc', refit=True, n_iter=25) # define random search
xgb_gs.fit(train_data['text_'], train_data['label']) # fit random search

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 12 is smaller than n_iter=25. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 1 folds for each of 12 candidates, totalling 12 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=1, tfidf__max_df=0.5, tfidf__analyzer=word 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=1, tfidf__max_df=0.5, tfidf__analyzer=word, score=0.819, total=  51.9s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   51.8s remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=2, tfidf__max_df=0.5, tfidf__analyzer=word 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=2, tfidf__max_df=0.5, tfidf__analyzer=word, score=0.831, total=  31.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.4min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.5,

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0, ..., -1, -1])),
                   error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('tfidf',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                      

In [53]:
xgb_gs.best_score_ # print best score

0.8312627495370527

- Best score of xgboost is 83% and it did worse than random forest

In [108]:
xgb_gs.best_params_ # print best parameters of xgboost

{'tfidf__stop_words': 'english',
 'tfidf__norm': 'l2',
 'tfidf__ngram_range': (1, 2),
 'tfidf__min_df': 30,
 'tfidf__max_df': 0.5,
 'tfidf__analyzer': 'word',
 'Model__min_child_weight': 10,
 'Model__gamma': 1}

## 3.4 Naive Bayes

In [47]:
naive_pipe = Pipeline([('tfidf', TfidfVectorizer()), ('mb', MultinomialNB())]) # pipline for naive bayes

# naive bayes paramters
naive_params = {
        'tfidf__analyzer':["word"], # 1
        #'tfidf__max_df':[0.5, 0.8, 0.95, 0.99], # 4
        'tfidf__min_df':[1, 2, 5, 10, 20, 30, 40, 100], # 6
        #'tfidf__max_features':[5000, 6000, 7000, 8000, 9000, 10000], # 5
        'tfidf__norm':["l2"], # 1
        'tfidf__ngram_range': [(1, 2), (1, 3)], # 2
        'tfidf__stop_words': ['english'] # 1
 
}


naive_gs = RandomizedSearchCV(naive_pipe, naive_params, cv = pds, verbose =100, scoring='roc_auc', refit=True, n_iter=60) # define random search for naive bayes
naive_gs.fit(train_data['text_'], train_data['label']) # fit the random search for naive bayes

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 16 is smaller than n_iter=60. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 1 folds for each of 16 candidates, totalling 16 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=1, tfidf__analyzer=word 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=1, tfidf__analyzer=word, score=0.862, total=   6.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.0s remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=1, tfidf__analyzer=word 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=1, tfidf__analyzer=word, score=0.862, total=   9.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.4s remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=2, tfidf__analyzer=word 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0, ..., -1, -1])),
                   error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('tfidf',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                      

In [48]:
naive_gs.best_score_ # print best score for naive bayes

0.8620148626326889

- Naive bayes best score is 86.2% and it is much faster than Xgboost and Random forest as Random forest and XGBoost are ensemble methods that build multiple decision trees and combine their predictions. These algorithms are typically more computationally intensive than MultinomialNB because they involve building and evaluating multiple decision trees. Naive bayes did better than Xgboost and Random forest. so I think it is the best algorithm for this problem

In [49]:
naive_gs.best_params_ # print best parameters for naive bayes

{'tfidf__stop_words': 'english',
 'tfidf__norm': 'l2',
 'tfidf__ngram_range': (1, 2),
 'tfidf__min_df': 1,
 'tfidf__analyzer': 'word'}

- I Think I should try an algorithm that take weights that vectorizer gives for each word in its consideration so I will try logistic regression I think it will do better.

# 3.5 Logistic Regression

In [65]:
from sklearn.linear_model import LogisticRegression
log_pip = Pipeline([('tfidf', TfidfVectorizer()), ('log', LogisticRegression())]) # pipline for naive bayes

# naive bayes paramters
naive_params = {
        'tfidf__analyzer':["word"], # 1
        'tfidf__max_df':[0.1, 0.5, 0.8, 0.95, 0.99], # 4
        'tfidf__min_df':[1, 2, 5, 10], # 4
        #'tfidf__max_features':[5000, 6000, 7000, 8000, 9000, 10000], # 5
        'tfidf__norm':["l2"], # 1
        'tfidf__ngram_range': [(1, 2), (1, 3)], # 2
        'tfidf__stop_words': ['english'], # 1
        'log__solver':['lbfgs', 'sag', 'saga', 'newton-cg']
 
}


log_gs = RandomizedSearchCV(log_pip, naive_params, cv = pds, verbose =100, scoring='roc_auc', refit=True, n_iter=60) # define random search for naive bayes
log_gs.fit(train_data['text_'], train_data['label']) # fit the random search for naive bayes

Fitting 1 folds for each of 60 candidates, totalling 60 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=1, tfidf__max_df=0.5, tfidf__analyzer=word, log__solver=lbfgs 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=1, tfidf__max_df=0.5, tfidf__analyzer=word, log__solver=lbfgs, score=0.863, total=  37.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.3s remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=10, tfidf__max_df=0.8, tfidf__analyzer=word, log__solver=sag 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=10, tfidf__max_df=0.8, tfidf__analyzer=word, log__solver=sag, score=0.871, total=  11.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   48.4s remaining:    0.0s
[CV] tfidf__stop_words=english, tfid

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=5, tfidf__max_df=0.99, tfidf__analyzer=word, log__solver=lbfgs, score=0.874, total=  14.2s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.2min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.99, tfidf__analyzer=word, log__solver=newton-cg 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.99, tfidf__analyzer=word, log__solver=newton-cg, score=0.874, total=   9.6s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.4min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.1, tfidf__analyzer=word, log__solver=lbfgs 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.1, tfidf__analyzer=word, log__solver=lbfgs, score=0.867,

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=2, tfidf__max_df=0.1, tfidf__analyzer=word, log__solver=lbfgs, score=0.868, total=   9.5s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.8min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=2, tfidf__max_df=0.99, tfidf__analyzer=word, log__solver=newton-cg 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=2, tfidf__max_df=0.99, tfidf__analyzer=word, log__solver=newton-cg, score=0.874, total=   9.4s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.9min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.99, tfidf__analyzer=word, log__solver=sag 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=5, tfidf__max_df=0.99, tfidf__analyzer=word, log__solver=sag, score=0.874, to

...

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 2), tfidf__min_df=10, tfidf__max_df=0.95, tfidf__analyzer=word, log__solver=lbfgs, score=0.871, total=  18.0s
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 12.4min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=5, tfidf__max_df=0.1, tfidf__analyzer=word, log__solver=sag 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=5, tfidf__max_df=0.1, tfidf__analyzer=word, log__solver=sag, score=0.867, total=  16.2s
[Parallel(n_jobs=1)]: Done  55 out of  55 | elapsed: 12.6min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=5, tfidf__max_df=0.1, tfidf__analyzer=word, log__solver=lbfgs 


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=5, tfidf__max_df=0.1, tfidf__analyzer=word, log__solver=lbfgs, score=0.867, total=  14.7s
[Parallel(n_jobs=1)]: Done  56 out of  56 | elapsed: 12.9min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=2, tfidf__max_df=0.5, tfidf__analyzer=word, log__solver=sag 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=2, tfidf__max_df=0.5, tfidf__analyzer=word, log__solver=sag, score=0.874, total=  15.3s
[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed: 13.1min remaining:    0.0s
[CV] tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=1, tfidf__max_df=0.95, tfidf__analyzer=word, log__solver=lbfgs 
[CV]  tfidf__stop_words=english, tfidf__norm=l2, tfidf__ngram_range=(1, 3), tfidf__min_df=1, tfidf__max_df=0.95, tfidf__analyzer=word, log__solver=lbfgs, score=0.863, total=  36.2

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0, ..., -1, -1])),
                   error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('tfidf',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                      

In [66]:
log_gs.best_score_

0.8744059107103052

- logistic regression gets an 87.4% score. the reason I think behind this because it takes the weights of number of each word in its consideration.

In [67]:
log_gs.best_params_

{'tfidf__stop_words': 'english',
 'tfidf__norm': 'l2',
 'tfidf__ngram_range': (1, 2),
 'tfidf__min_df': 2,
 'tfidf__max_df': 0.5,
 'tfidf__analyzer': 'word',
 'log__solver': 'sag'}

In [68]:
test_data_orig = pd.read_csv('x_test.csv') # reading the test data
test_data = data_preprocessing(test_data_orig.copy()) # do the preprocessing

submission = pd.DataFrame() # create a dataframe
submission['id'] = test_data_orig['id'] # get the id column

submission['label'] = log_gs.predict_proba(test_data['text_'])[:,1] # predict the values

submission.to_csv('sample_submission_walkthrough.csv', index=False) # save the result

# 5. Conculsion

- Preprocessing done using lemmatization
- we used different approch in vectorizer which is word and character analyzer and we use different n-grams
- (1, 2) word analyzer is much better than (1, 2), (1, 3), (1, 4) character analyzer for this problem
- we used different models like random forest, xgboost and naive bayes
- Naive bayes best score is 86.2% and it is much faster than Xgboost and Random forest as Random forest and XGBoost as Random forest and XGBoost are ensemble methods
- logistic regression gets an 87.4% score. the reason I think behind this because it takes the weights of number of each word in its consideration and it's faster than Xgboost and Random forest as Random forest so I think it is the suitable one among other algorithms.

# 6. References:
- https://enjoymachinelearning.com/blog/countvectorizer-vs-tfidfvectorizer/
- https://www.machinelearningplus.com/nlp/what-is-tokenization-in-natural-language-processing/
- https://nlp.stanford.edu/IR-book/html/htmledition/tokenization-1.html